In [1]:
import os
import numpy as np
import pandas as pd
import random
import itertools
import pickle
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from tqdm import tqdm

from numpy.random import RandomState
from sklearn.utils import shuffle

import argparse
import seaborn as sns

In [3]:
###############################################
#### Input dataset name
###############################################
root_folder = './datasets/scenario23_dev/'
data_csv = root_folder + 'scenario23.csv'




###############################################
# Read dataset to create a list of the input sequence   
###############################################

df = pd.read_csv(data_csv)
image_data_lst = df['unit1_rgb'].values
pwr_data_lst = df['unit1_pwr_60ghz'].values
#original_beam = df['unit1_beam_index'].values


###############################################
#### subsample the power and generate the 
#### updated beam indices
###############################################
updated_beam = []
original_beam = []
for entry in pwr_data_lst:
    data_to_read = f'./{root_folder}{entry[1:]}'
    pwr_data = np.loadtxt(data_to_read)
    original_beam.append(np.argmax(pwr_data)+1)
    updated_pwr = []
    j = 0
    while j < (len(pwr_data)- 1):
        tmp_pwr = pwr_data[j]
        updated_pwr.append(tmp_pwr)
        j += 2
    updated_beam.append(np.argmax(updated_pwr)+1)
    






In [4]:
def create_img_beam_dataset():   
    
    folder_to_save = 'image_beam'
    if not os.path.exists(folder_to_save):
        os.makedirs(folder_to_save)
    
    #############################################
    ###### created updated image path ###########
    #############################################
    updated_img_path = []
    print("Creating the image based dataset")
    for entry in tqdm(image_data_lst):
        img_path = entry.split('./')[1]
        updated_path = f'./{root_folder}/{img_path}'
        updated_img_path.append(updated_path)
    
    
    #############################################
    # saving the image-beam development dataset for training and validation
    #############################################
                            
    indx = np.arange(1, len(updated_beam)+1,1)
    df_new = pd.DataFrame()
    df_new['index'] = indx   
    df_new['unit1_rgb'] = updated_img_path   
    df_new['unit1_beam'] = updated_beam    
    df_new.to_csv(fr'./{folder_to_save}/scenario23_img_beam.csv', index=False)       
      
    #############################################
    #generate the train and test dataset
    #############################################    
    rng = RandomState(1)
    train, val, test = np.split(df_new.sample(frac=1, random_state=rng ), [int(.6*len(df_new)), int(.9*len(df_new))])
    train.to_csv(f'./{folder_to_save}/scenario23_img_beam_train.csv', index=False)
    val.to_csv(f'./{folder_to_save}/scenario23_img_beam_val.csv', index=False)
    test.to_csv(f'./{folder_to_save}/scenario23_img_beam_test.csv', index=False)


def create_pos_beam_dataset():  
    
    folder_to_save = './pos_beam'
    if not os.path.exists(folder_to_save):
        os.makedirs(folder_to_save)
    
    ###############################################
    ####### read position values from dataset #####
    ###############################################

    lat = []
    lon = []
    pos_data_path = df['unit2_loc'].values
    print("Creating the position based dataset")
    print("Reading the dataset")
    for entry in tqdm(pos_data_path):
        data_to_read = f'./{root_folder}{entry[1:]}'
        pos_val = np.loadtxt(data_to_read)
        #lat_val, lon_val = pos_val[0], pos_val[1]
        lat.append(pos_val[0])
        lon.append(pos_val[1])
        
    def norm_data(data_lst):
        norm_data = []
        for entry in data_lst:
            norm_data.append((entry - min(data_lst))/(max(data_lst) - min(data_lst)))
        return norm_data

    ###############################################
    ##### normalize latitude and longitude data ###
    ###############################################
    lat_norm = norm_data(lat)
    lon_norm = norm_data(lon)

    ###############################################
    ##### generate final pos data #################
    ###############################################
    pos_data = []
    for j in range(len(lat_norm)):
        pos_data.append([lat_norm[j], lon_norm[j]])


    #############################################
    # saving the pos-beam development dataset for training and validation
    #############################################
                            
    indx = np.arange(1, len(updated_beam)+1,1)
    df_new = pd.DataFrame()
    df_new['index'] = indx   
    df_new['unit2_pos'] = pos_data   
    df_new['unit1_beam'] = updated_beam    
    df_new.to_csv(fr'./{folder_to_save}/scenario23_pos_beam.csv', index=False) 
    
    #############################################
    #generate the train and test dataset
    #############################################    
    rng = RandomState(1)
    train, val, test = np.split(df_new.sample(frac=1, random_state=rng ), [int(.6*len(df_new)), int(.9*len(df_new))])
    train.to_csv(f'./{folder_to_save}/scenario23_pos_beam_train.csv', index=False)
    val.to_csv(f'./{folder_to_save}/scenario23_pos_beam_val.csv', index=False)
    test.to_csv(f'./{folder_to_save}/scenario23_pos_beam_test.csv', index=False)






create_img_beam_dataset()
create_pos_beam_dataset()

Creating the image based dataset


100%|██████████| 11387/11387 [00:00<00:00, 2186837.90it/s]
C:\Users\abdu_\anaconda3\envs\sionna_env\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Creating the position based dataset
Reading the dataset


100%|██████████| 11387/11387 [01:24<00:00, 134.27it/s]
C:\Users\abdu_\anaconda3\envs\sionna_env\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
def create_pos_height_beam_dataset():  
    
    folder_to_save = './pos_height_beam'
    if not os.path.exists(folder_to_save):
        os.makedirs(folder_to_save)
    
    ###############################################
    ####### read position values from dataset #####
    ###############################################

    lat = []
    lon = []
    height = []
    distance = []
    
    pos_data_path = df['unit2_loc'].values
    height_data_path = df['unit2_height'].values
    distance_data_path = df['unit2_distance'].values
    print("Creating the position based dataset")
    print("Reading the dataset")
    for entry in tqdm(pos_data_path):
        data_to_read = f'./{root_folder}{entry[1:]}'
        pos_val = np.loadtxt(data_to_read)
        #lat_val, lon_val = pos_val[0], pos_val[1]
        lat.append(pos_val[0])
        lon.append(pos_val[1])
        
    for entry_height, entry_distance in zip(tqdm(height_data_path), distance_data_path) :
        data_to_read_height = f'./{root_folder}{entry_height[1:]}'
        data_to_read_distance = f'./{root_folder}{entry_distance[1:]}'
        
        height_val = np.loadtxt(data_to_read_height)
        distance_val = np.loadtxt(data_to_read_distance)
        height.append(height_val)
        distance.append(distance_val)
        
        
    def norm_data(data_lst):
        norm_data = []
        for entry in data_lst:
            norm_data.append((entry - min(data_lst))/(max(data_lst) - min(data_lst)))
        return norm_data

    ###############################################
    ##### normalize latitude and longitude data ###
    ###############################################
    lat_norm = norm_data(lat)
    lon_norm = norm_data(lon)
    height_norm = norm_data(height)
    distance_norm = norm_data(distance)
    ###############################################
    ##### generate final pos data #################
    ###############################################
    #pos_data = []
    #for j in range(len(lat_norm)):
    #    pos_data.append([lat_norm[j], lon_norm[j]])


    #############################################
    # saving the pos-beam development dataset for training and validation
    #############################################
                            
    indx = np.arange(1, len(updated_beam)+1,1)
    df_new = pd.DataFrame()
    df_new['index'] = indx   
    df_new['unit2_lat'] = lat_norm
    df_new['unit2_lon'] = lon_norm
    df_new['unit2_height'] = height_norm
    df_new['unit2_distance'] = distance_norm
    df_new['unit1_beam'] = updated_beam    
    df_new.to_csv(fr'./{folder_to_save}/scenario23_pos_height_beam.csv', index=False) 
    
    #############################################
    #generate the train and test dataset
    #############################################    
    rng = RandomState(1)
    train, val, test = np.split(df_new.sample(frac=1, random_state=rng ), [int(.6*len(df_new)), int(.9*len(df_new))])
    train.to_csv(f'./{folder_to_save}/scenario23_pos_height_beam_train.csv', index=False)
    val.to_csv(f'./{folder_to_save}/scenario23_pos_height_beam_val.csv', index=False)
    test.to_csv(f'./{folder_to_save}/scenario23_pos_height_beam_test.csv', index=False)   



create_pos_height_beam_dataset()

Creating the position based dataset
Reading the dataset


100%|██████████| 11387/11387 [01:43<00:00, 109.73it/s]
C:\Users\abdu_\anaconda3\envs\sionna_env\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
###############################################
#### Input dataset name
###############################################
root_folder = './datasets/scenario23_dev/'
data_csv = root_folder + 'scenario23.csv'




###############################################
# Read dataset to create a list of the input sequence   
###############################################

df = pd.read_csv(data_csv)
image_data_lst = df['unit1_rgb'].values
pwr_data_lst = df['unit1_pwr_60ghz'].values
#original_beam = df['unit1_beam_index'].values


###############################################
#### subsample the power and generate the 
#### updated beam indices
###############################################
updated_beam = []
original_beam = []
for num, entry in zip(range(5), pwr_data_lst):
    data_to_read = f'./{root_folder}{entry[1:]}'
    pwr_data = np.loadtxt(data_to_read)
    original_beam.append(np.argmax(pwr_data)+1)
    updated_pwr = []
    j = 0
    print(data_to_read)
    if os.path.exists(data_to_read):
        print("Exist")
    while j < (len(pwr_data)- 1):
        tmp_pwr = pwr_data[j]
        updated_pwr.append(tmp_pwr)
        j += 2
    updated_beam.append(np.argmax(updated_pwr)+1)

././datasets/scenario23_dev//unit1/mmWave_data/mmWave_power_1.txt
Exist
././datasets/scenario23_dev//unit1/mmWave_data/mmWave_power_2.txt
Exist
././datasets/scenario23_dev//unit1/mmWave_data/mmWave_power_3.txt
Exist
././datasets/scenario23_dev//unit1/mmWave_data/mmWave_power_4.txt
Exist
././datasets/scenario23_dev//unit1/mmWave_data/mmWave_power_5.txt
Exist


In [7]:
df = pd.read_csv(data_csv)
image_data_lst = df['unit1_rgb'].values
pwr_data_lst = df['unit1_pwr_60ghz'].values

updated_img_path = []
print("Creating the image based dataset")
for num, entry in zip(range(5), image_data_lst):
    img_path = entry.split('./')[1]
    updated_path = f'./{root_folder}/{img_path}'
    updated_img_path.append(updated_path)
    print(img_path)
    print(updated_path)
    if os.path.exists(updated_path):
        print("Exist")

Creating the image based dataset
unit1/camera_data/image_BS1_1_16_58_42.jpg
././datasets/scenario23_dev//unit1/camera_data/image_BS1_1_16_58_42.jpg
Exist
unit1/camera_data/image_BS1_2_16_58_42.jpg
././datasets/scenario23_dev//unit1/camera_data/image_BS1_2_16_58_42.jpg
Exist
unit1/camera_data/image_BS1_3_16_58_42.jpg
././datasets/scenario23_dev//unit1/camera_data/image_BS1_3_16_58_42.jpg
Exist
unit1/camera_data/image_BS1_4_16_58_42.jpg
././datasets/scenario23_dev//unit1/camera_data/image_BS1_4_16_58_42.jpg
Exist
unit1/camera_data/image_BS1_5_16_58_42.jpg
././datasets/scenario23_dev//unit1/camera_data/image_BS1_5_16_58_42.jpg
Exist
